In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from utils.topo_utils import calculate_circumcenters_torch, compute_circumsphere_jacobian

# Fixed vertices on the unit sphere.
# p0, p1, and p2 define a plane.
p0 = torch.tensor([0.0, 0.0, 1.0])
p1 = torch.tensor([0.0, 1.0, 0.0])
p2 = torch.tensor([1.0, 0.0, 0.0])

# p3_0 is chosen to lie exactly in the plane defined by p0, p1, and p2.
p3_0 = torch.tensor([2/3, 2/3, -1/3])

# For the plane x+y+z=1, the (unnormalized) normal is (1,1,1)
normal = torch.tensor([1.0, 1.0, 1.0])
normal = normal / torch.norm(normal)

def get_p3(t: float) -> torch.Tensor:
    """
    Perturb p3_0 along the normal by a small amount t,
    then renormalize to project back onto the unit sphere.
    When t = 0, p3 would be exactly coplanar with p0, p1, and p2.
    """
    pt = p3_0 + t * normal
    return pt / torch.norm(pt)

# Use a small delta for finite-difference (simulating a tiny perturbation).
delta = 1e-6

# Use a log-spaced range of t values from 1e-1 to 1e-4 to see how the sensitivity increases
t_values = np.logspace(-1, -4, num=50)
sensitivities = []
finite_diff_changes = []
predicted_changes = []
centers = []

for t in t_values:
    p3_t = get_p3(t)
    vertices = torch.stack([p0, p1, p2, p3_t], dim=0)
    
    # Compute the circumcenter.
    center = compute_circumcenter(vertices)
    centers.append(center.numpy())
    
    # Compute the Jacobian and extract the block for p3.
    jacobian = compute_circumsphere_jacobian(vertices)
    jacobian_p3 = jacobian[:, 3, :]
    sens = torch.linalg.norm(jacobian_p3)
    sensitivities.append(sens.item())
    
    # Apply a small finite-difference perturbation along the normal to p3.
    vertices_perturbed = vertices.clone()
    vertices_perturbed[3] = vertices[3] + delta * normal
    vertices_perturbed[3] = vertices_perturbed[3] / torch.norm(vertices_perturbed[3])
    
    center_perturbed = compute_circumcenter(vertices_perturbed)
    fd_change = torch.linalg.norm(center_perturbed - center)
    finite_diff_changes.append(fd_change.item())
    
    predicted_changes.append(sens.item() * delta)

centers = np.array(centers)

# Create the plots.
plt.figure(figsize=(12, 6))

# Plot 1: Sensitivity, finite difference change, and predicted change versus t.
plt.subplot(1, 2, 1)
plt.semilogx(t_values, sensitivities, label='Sensitivity', marker='o')
plt.semilogx(t_values, finite_diff_changes, label='Finite Difference Change', marker='s')
plt.semilogx(t_values, predicted_changes, label='Predicted Change (sens*delta)', linestyle='--')
plt.xlabel("t (perturbation magnitude)")
plt.ylabel("Value")
plt.title("Sensitivity & Finite Difference Change vs. t")
plt.legend()
plt.grid(True)

# Plot 2: Evolution of the circumcenter coordinates versus t.
plt.subplot(1, 2, 2)
plt.semilogx(t_values, centers[:, 0], label='Circumcenter X', marker='o')
plt.semilogx(t_values, centers[:, 1], label='Circumcenter Y', marker='s')
plt.semilogx(t_values, centers[:, 2], label='Circumcenter Z', marker='^')
plt.xlabel("t (perturbation magnitude)")
plt.ylabel("Circumcenter Coordinate")
plt.title("Circumcenter Coordinates vs. t")
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
